# SDC Ratio and SDC Impact

SDC ratio is a stable metrics. If fault injection campaign sample the same program many time and calculate the SDC ratio, for each campaign the SDC ratio will be the same.

SDC Impact is a unstable metrics. For each fault injection campaign program's expected SDC impact may change a lot.

In [ ]:
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy.stats import iqr

In [ ]:
cg_df = pd.read_csv("../static/data/cg_in8.csv")
#cg_df = pd.read_csv("fft_exhaust.csv")

highbit_dataset = cg_df[cg_df.bit > 32]
exp_dataset = cg_df[cg_df.bit > 52]
mantissa_dataset = cg_df[cg_df.bit < 53]

In [ ]:
count = 0 
for i in range(int(len(cg_df)/64)):
    
    max_value = float('inf')
    min_value = float('-inf')
    for j in range(64):
        if cg_df.outcome[i * 64 + j] == "SDC":
            if cg_df.out_xor[i * 64 + j] < max_value and cg_df.out_xor[i * 64 + j] > 0:
                max_value = cg_df.out_xor[i * 64 + j]
                
            if cg_df.out_xor[i*64 + j] > min_value and cg_df.out_xor[i * 64 + j] < 0:
                min_value = cg_df.out_xor[i * 64 + j]
    
    for j in range(64):      
        if cg_df.outcome[i * 64 + j] == 'Masked':
           
            if cg_df.out_xor[i * 64 + j] > max_value or cg_df.out_xor[i * 64 + j] < min_value:
                #print(cg_df.Line[i * 64 + j])
                count += 1

In [ ]:
s = np.random.poisson(1, 10000)
count, bins, ignored = plt.hist(s, 5, density=True)

print(np.max(s))

plt.show()

In [ ]:
def sampleFunction(dataset, sample_size):
    return dataset.sample(n=sample_size)

def samplePissionDistribution(dataset, sample_size):
    s = np.random.poisson(1, sample_size)
    
    
    pass

def iteractiveSampleFunction(dataset, sample_size, filter_bit):
    filter_dataset = dataset[dataset.bit > filter_bit-1]
    return filter_dataset.sample(n=int(sample_size))

def iteractiveSampling(dataset, sample_size, iteration=10):
    sample_size_each_iter = sample_size/iteration
    sample = []
    
    mantissa_lowest_SDC_bit = 1
    mantissa_highest_bit = 64
    lowest_SDC_bit = 0
    
    for i in range(iteration):
        
        current_iter_sample = iteractiveSampleFunction(dataset, sample_size_each_iter, mantissa_lowest_SDC_bit)
        current_lowest_SDC_bit = current_iter_sample[current_iter_sample.outcome == "SDC"]["bit"].min()
        
        print(current_lowest_SDC_bit)
        sample.append((mantissa_lowest_SDC_bit, current_iter_sample))
        
        #check the lowest impact bit
        if(lowest_SDC_bit < current_lowest_SDC_bit):
            lowest_SDC_bit = current_lowest_SDC_bit
        
        if(lowest_SDC_bit > int((mantissa_highest_bit + mantissa_lowest_SDC_bit)/2)):
            mantissa_lowest_SDC_bit = int((mantissa_highest_bit + mantissa_lowest_SDC_bit)/2)
        else:
            mantissa_lowest_SDC_bit = lowest_SDC_bit
    return sample

In [ ]:
def calculateSDCratio(df, line, rescale_ratio=1):
    temp = df[df.Line == line]
    numberOfFaultInjection = len(temp)
    numberOfSDC = len(temp[temp.outcome == "SDC"])

    if numberOfFaultInjection is not 0: 
        return float(numberOfSDC)/numberOfFaultInjection * rescale_ratio
    else:
        return 0
    
def calculateExpectedSDCImpact(df, line, rascale_ratio=1):
    temp = df[df.Line == line]
    temp = temp[temp.diffnormr != float("inf")]
    temp = temp[temp.diffnormr != float("-inf")]
    temp = temp[temp.diffnormr.notnull()]

    if len(temp) == 0:
        return 0
    else:
        temp = temp.diffnormr.abs()
    
    l = len(temp)
    temp = np.sum(temp/l)
    
    if temp * rascale_ratio < 1:
        return temp * rascale_ratio
    
    return np.log(temp * rascale_ratio)

def calculateSDCratio_iteractiveSampling(data, line):

    totalnumberofSDC = 0
    totalFaultInjection = 0
    for sample in data:
        temp = sample[1][sample[1].Line == line]
        numberOfFaultInjection = len(temp)
        numberOfSDC = len(temp[temp.outcome == "SDC"])
        
        totalnumberofSDC += numberOfSDC
        totalFaultInjection += (numberOfFaultInjection * 64.0/(64.0 - sample[0] + 1.0))

    if totalnumberofSDC is 0:
        return 0
    else:
        return float(totalnumberofSDC)/totalFaultInjection


def calculateSDCImpact_iteractiveSampling(data, line):
    
    SDC_of_each_iteration = []
    for sample in data:
        temp = sample[1][sample[1].Line == line]
        temp = temp[temp.diffnormr != float("inf")]
        temp = temp[temp.diffnormr != float("-inf")]
        temp = temp[temp.diffnormr.notnull()]
        
        if len(temp) is 0:
            SDC_of_each_iteration.append(0)
        else:
            temp = temp.diffnormr.abs()
            l = len(temp)
            temp = np.sum(temp/l)
            
            SDC_of_each_iteration.append(temp * ((64.0-sample[0]+1)/64.0))
    
    if np.sum(SDC_of_each_iteration) < 1:
        return np.sum(SDC_of_each_iteration)
    #print np.sum(SDC_of_each_iteration)
    
    return np.log(np.sum(SDC_of_each_iteration))

each_line_sdc_ratio_distribution_random = {}
each_line_sdc_ratio_distribution_hightbit = {}
each_line_sdc_ratio_distribution_iterative = {}

each_line_expected_sdc_impact_distribution_random = {}
each_line_expected_sdc_impact_distribution_highbit= {}
each_line_expected_sdc_impact_distribution_iterative = {}

lines = []
line_sample = list(cg_df.Line.unique())

lines = line_sample

    
for i in range(1):
    exp_sample = sampleFunction(exp_dataset, int(1000 * 0.8))
    mantissa_sample = sampleFunction(mantissa_dataset, int(1000 * 0.2))
    random_sample = sampleFunction(cg_df, 1000)
    iteractive_sample = iteractiveSampling(cg_df, 1000)
    
    iteractive = pd.DataFrame(iteractive_sample[0][1])
    
    for k in range(1, len(iteractive_sample)):
        iteractive_sample[k][1]
        iteractive = pd.concat([iteractive, iteractive_sample[k][1]])

    #print(iteractive)
    #pd.concat([exp_sample, mantissa_sample]).to_csv("cg_8_2.csv", index = None)
    #random_sample.to_csv("cg_random.csv", index = None)
    iteractive.to_csv("cg_iteractive.csv", index = None)
    #print(iteractive_sample[0])
    
    for line in lines:
        #init the dictionary
        if line not in each_line_sdc_ratio_distribution_random:
            #SDC ratio
            each_line_sdc_ratio_distribution_random[line] = []
            each_line_sdc_ratio_distribution_hightbit[line] = []
            each_line_sdc_ratio_distribution_iterative[line] = []
            
            #SDC impact
            each_line_expected_sdc_impact_distribution_random[line] = []
            each_line_expected_sdc_impact_distribution_highbit[line] = []
            each_line_expected_sdc_impact_distribution_iterative[line] = []
            
        #SDC ratio
        each_line_sdc_ratio_distribution_random[line].append(calculateSDCratio(random_sample, line))
        each_line_sdc_ratio_distribution_hightbit[line].append(calculateSDCratio(exp_sample, line) * 12.0/64.0 + calculateSDCratio(mantissa_sample, line) * 52.0/64.0)
        each_line_sdc_ratio_distribution_iterative[line].append(calculateSDCratio_iteractiveSampling(iteractive_sample, line))
        
        #SDC impact
        each_line_expected_sdc_impact_distribution_random[line].append(calculateExpectedSDCImpact(random_sample, line))
        each_line_expected_sdc_impact_distribution_highbit[line].append(calculateExpectedSDCImpact(exp_sample, line, 12.0/64.0)  + calculateExpectedSDCImpact(mantissa_sample, line, 52.0/64.0) )
        each_line_expected_sdc_impact_distribution_iterative[line].append(calculateSDCImpact_iteractiveSampling(iteractive_sample, line))


# SDC Ratio and SDC Impact

In [ ]:
# SDC ratio
random_expected_sdc = []
random_var_sdc = []

highbit_expected_sdc= []
highbit_var_sdc = []

iteractive_expected_sdc = []
iteractive_var_sdc = []

golden_sdc = []
golden_sdc_label = []

#SDC impact
random_expected_impact = []
random_var_impact = []

highbit_expected_impact = []
highbit_var_impact = []

iteractive_expected_impact = []
iteractive_var_impact = []

golden_impact = []

for line in lines:

    #SDC ratio
    random_expected_sdc.append(np.mean(each_line_sdc_ratio_distribution_random[line]))
    random_var_sdc.append(np.var(each_line_sdc_ratio_distribution_random[line]))
    
    highbit_expected_sdc.append(np.mean(each_line_sdc_ratio_distribution_hightbit[line]))
    highbit_var_sdc.append(np.var(each_line_sdc_ratio_distribution_hightbit[line]))
    
    iteractive_expected_sdc.append(np.mean(each_line_sdc_ratio_distribution_iterative[line]))
    iteractive_var_sdc.append(np.var(each_line_sdc_ratio_distribution_iterative[line]))
    
    golden_sdc.append(calculateSDCratio(cg_df, line))
    
    #SDC impact
    random_expected_impact.append(np.mean(each_line_expected_sdc_impact_distribution_random[line]))
    random_var_impact.append(np.var(each_line_expected_sdc_impact_distribution_random[line]))
    
    highbit_expected_impact.append(np.mean(each_line_expected_sdc_impact_distribution_highbit[line]))
    highbit_var_impact.append(np.var(each_line_expected_sdc_impact_distribution_highbit[line]))
    
    iteractive_expected_impact.append(np.mean(each_line_expected_sdc_impact_distribution_iterative[line]))
    iteractive_var_impact.append(np.var(each_line_expected_sdc_impact_distribution_iterative[line]))
    
    golden_impact.append(calculateExpectedSDCImpact(cg_df, line))
    golden_sdc_label.append(line)

# SDC Ratio 

In [ ]:
data = []

def parseDataToFrameFormat(data, category):
    arrays = []
    for key in data:
        for value in data[key]:
            arrays.append([key, value, category])
    return arrays 

data = data + parseDataToFrameFormat(each_line_sdc_ratio_distribution_random, 'Random')
data = data + parseDataToFrameFormat(each_line_sdc_ratio_distribution_hightbit, 'Highbit')
data = data + parseDataToFrameFormat(each_line_sdc_ratio_distribution_iterative, 'Iteractive')

for i in range(len(golden_sdc_label)):
    data.append([golden_sdc_label[i], golden_sdc[i], 'golden'])

sns.set(style="whitegrid")

df = pd.DataFrame(data, columns=['Line', 'SDC Ratio', 'Category'])
#print (df)
plt.title('Sampling Methods Test on Conjugate Gradient Dataset')

ax = sns.barplot(x="Line", y="SDC Ratio", hue="Category", data=df, errwidth=1, capsize=0.15, ci=68)
#sns.palplot(sns.color_palette("hls", 8))
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

# SDC Impact

In [ ]:
data = []

def parseDataToFrameFormat(data, category):
    arrays = []
    for key in data:
        for value in data[key]:
            arrays.append([key, value, category])
    return arrays 

data = data + parseDataToFrameFormat(each_line_expected_sdc_impact_distribution_random, 'Random')
data = data + parseDataToFrameFormat(each_line_expected_sdc_impact_distribution_highbit, 'Highbit')
data = data + parseDataToFrameFormat(each_line_expected_sdc_impact_distribution_iterative, 'Iterative')

for i in range(len(golden_sdc_label)):
    data.append([golden_sdc_label[i], golden_impact[i], 'golden'])

df = pd.DataFrame(data, columns=['Line', 'SDC Impact', 'Category'])

plt.title('Sampling Methods Test on Conjugate Gradient Dataset')
sns.set(style="whitegrid")
ax = sns.barplot(x="Line", y="SDC Impact", hue="Category", data=df, errwidth=1, capsize=0.2, ci=68)
#ax.get_legend().remove()
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()